In [2]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
import math
from tensorflow.keras import layers
from tensorflow import keras

/Users/ajayvikram/MachineLearning/venv/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [3]:
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [5]:
(ds_train, ds_test), ds_info  = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=False,
    as_supervised=True,
    with_info=True 
)

@tf.function
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

@tf.function
def rotate(img, max_degrees=25):
    degrees = tf.random.uniform([], -max_degrees, max_degrees, dtype=tf.float32)
    img = tfa.image.rotate(img, degrees*math.pi/180, interpolation="BILINEAR")
    return img

@tf.function
def augment(image, label):
    image = tf.image.resize(image, size=[28, 28])
    image = rotate(image)

    # random coloring of image
    image =tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, lower=0.5, upper=1.5)

    return image, label


AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits["train"].num_examples)
ds_train = ds_train.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.map(augment, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.batch(BATCH_SIZE)
ds_train = ds_train.prefetch(AUTOTUNE)

ds_test = ds_test.map(normalize_img, num_parallel_calls=AUTOTUNE)
ds_test = ds_test.batch(BATCH_SIZE)
ds_test = ds_test.prefetch(AUTOTUNE)

def my_model():
    inputs = keras.Input(shape=(28, 28, 1))
    x = layers.Conv2D(32, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(128, 3)(inputs)
    x = layers.BatchNormalization()(x)
    x = keras.activations.relu(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(64, activation="softmax")(x)
    return keras.Model(inputs=inputs, outputs=outputs)

In [8]:
model = my_model()
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    optimizer = keras.optimizers.legacy.Adam(learning_rate=1e-4),
    metrics = ["accuracy"],
)

In [10]:
model.fit(ds_train, epochs=30, verbose=1)

Epoch 1/30
1875/1875 [==============================] - 65s 35ms/step - loss: 0.1983 - accuracy: 0.9444
Epoch 2/30
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1560 - accuracy: 0.9573
Epoch 3/30
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1326 - accuracy: 0.9628
Epoch 4/30
1875/1875 [==============================] - 68s 36ms/step - loss: 0.1273 - accuracy: 0.9656
Epoch 5/30
1875/1875 [==============================] - 67s 36ms/step - loss: 0.1151 - accuracy: 0.9685
Epoch 6/30
1875/1875 [==============================] - 69s 37ms/step - loss: 0.1042 - accuracy: 0.9711
Epoch 7/30
1875/1875 [==============================] - 70s 37ms/step - loss: 0.1045 - accuracy: 0.9709
Epoch 8/30
1875/1875 [==============================] - 70s 37ms/step - loss: 0.1002 - accuracy: 0.9726
Epoch 9/30
1875/1875 [==============================] - 67s 36ms/step - loss: 0.0934 - accuracy: 0.9751
Epoch 10/30
1875/1875 [==============================] - 66s 35m

In [11]:
model.evaluate(ds_test)

313/313 [==============================] - 8s 25ms/step - loss: 0.0711 - accuracy: 0.9804


[0.07114478200674057, 0.980400025844574]

In [12]:
model.save("model")

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


In [13]:
import tensorflow as tf

saved_dir = "model"
converter = tf.lite.TFLiteConverter.from_saved_model(saved_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
model_lite = converter.convert()

with open("model.tflite", "wb") as f:
    f.write(model_lite)